In [1]:
import polars as pl
import pandas as pd
import numpy as np
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import catboost as cb
import sgml, sgpp

/home/sun9sun9/python312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sklearn.metrics import roc_auc_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder

skf = StratifiedKFold(5, random_state = 123, shuffle = True)
ss = StratifiedShuffleSplit(1, train_size = 0.8, random_state = 123)
ss_v = StratifiedShuffleSplit(1, train_size = 0.9, random_state = 123)

In [9]:
p = make_pipeline(
    sgpp.PolarsProcessor(predefined_types={'id': pl.Int64}),
    sgpp.ExprProcessor({
        'loan_paid_back': pl.col('loan_paid_back').cast(pl.Int8)
    }),
    sgpp.PandasConverter(index_col = 'id')
)
df_train = p.fit_transform('data/train.csv')
df_test = p.transform('data/test.csv')
with open('grade_subgrade.pkl', 'rb') as f:
    c_map = pkl.load(f)
df_train['grade_subgrade_no'] = df_train['grade_subgrade'].map(c_map).astype('int')
df_test['grade_subgrade_no'] = df_test['grade_subgrade'].map(c_map).astype('int')
df_train.shape, df_test.shape

((593994, 13), (254569, 12))

In [6]:
X_all = df_test.columns.tolist()
X_num = ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'grade_subgrade_no']
X_cat = ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose']
y = 'loan_paid_back'

In [6]:
ShuffleSplit?

Init signature:
ShuffleSplit(
    n_splits=10,
    *,
    test_size=None,
    train_size=None,
    random_state=None,
)
Docstring:     
Random permutation cross-validator.

Yields indices to split data into training and test sets.

Note: contrary to other cross-validation strategies, random splits
do not guarantee that test sets across all folds will be mutually exclusive,
and might include overlapping samples. However, this is still very likely for
sizeable datasets.

Read more in the :ref:`User Guide <ShuffleSplit>`.

For visualisation of cross-validation behaviour and
comparison between common scikit-learn split methods
refer to :ref:`sphx_glr_auto_examples_model_selection_plot_cv_indices.py`

Parameters
----------
n_splits : int, default=10
    Number of re-shuffling & splitting iterations.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absol

In [7]:
class CachedSplitter():
    def __init__(X, y = None, sp = ShuffleSplit(n_splits = 1, random_state=1), sp_v = None, groups = None):
        self.train_idx_list = list()
        self.valid_idx_list = list()
        if type(X) == pd.DataFrame:
            idx_func = lambda x: {'X': X.iloc[x], 'y': y.iloc[x] if y is not None else None, 'groups': (groups.iloc[x] if groups is not None else None)}
        else:
            idx_func = lambda x: {'X': X[x], 'y': y[x] if y is not None else None, 'groups': (groups[x] if groups is not None else None)}
        for train_idx, valid_idx in sp.split(X, y, groups = groups):
            if sp_v is not None:
                self.train_idx_list.append([
                    (train_idx[train_v_idx], train_idx[valid_v_idx])
                    for train_v_idx, valid_v_idx in sp_v.split(**idx_func(train_idx))
                ])
            else:
                self.train_idx_list.append([
                    (train_idx, None)
                ])
            
            self.valid_idx_list.append(valid_idx)
    
    def split(*args, split_train = True):
        if type(args[0]) in [pd.DataFrame, pd.Series]:
            idx_func = lambda x: (i.iloc[x] for i in args)
        else:
            idx_func = lambda x: (i[x] for i in args)
        if split_train:
            for train_idx_list, valid_idx in zip(train_idx_list, valid_idx_list):
                yield [
                    (idx_func(train_idx), None if train_v_idx is None else idx_func(train_v_idx))
                    for train_idx, train_v_idx in train_idx_list
                ], idx_func(valid_idx)
        else:
            for train_idx_list, valid_idx in zip(train_idx_list, valid_idx_list):
                yield [
                    (idx_func(train_idx) if train_v_idx is None else idx_func(np.hstack([train_idx, train_v_idx])), None)
                    for train_idx, train_v_idx in train_idx_list
                ], idx_func(valid_idx)
            
def split_and_call(func, train_idx_list, valid_idx_list, *args, split_train = True):
    return [
        [func(train, train_v, valid) for train, train_v in train_list]
        for train_list, valid in split(train_idx_list, valid_idx_list, *args, split_train=split_train)
    ]

In [8]:
train_idx_list, valid_idx_list = get_split_idx(df_train[X_all].values, df_train[y].values, skf, ss_v)

In [149]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from functools import partial

def fit(func, X = None):
    def ret(train, *args):
        train = list(train)
        if len(train) > 1:
            return func().fit(train[0][X] if X is not None else train[0], train[1])
        else:
            return func().fit(train[0][X] if X is not None else train[0])
    return ret

In [97]:
std_list = split_and_call(
    fit(lambda : StandardScaler().set_output(transform = 'pandas'), X_num), 
    train_idx_list, valid_idx_list, df_train[X_all], df_train[y], split_train=False
)
std_list

[[StandardScaler()],
 [StandardScaler()],
 [StandardScaler()],
 [StandardScaler()],
 [StandardScaler()]]

In [98]:
std_list[0][0].get_feature_names_out(X_num)

array(['annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate', 'grade_subgrade_no'], dtype=object)

In [167]:
pca_list = split_and_call(
    fit(lambda : make_pipeline(StandardScaler(), PCA()).set_output(transform = 'pandas'), X_num), 
    train_idx_list, valid_idx_list, df_train[X_all], df_train[y], split_train=False
)
pca_list

[[Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA())])],
 [Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA())])],
 [Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA())])],
 [Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA())])],
 [Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA())])]]

In [100]:
pca_list[0][0].get_feature_names_out(X_num)

array(['pca0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5'], dtype=object)

In [203]:
def split_and_transform(obj_lists, train_idx_list, valid_idx_list, *args, split_train=False):
    def get_feature_names(i, j):
        names = list()
        for name, obj_list, X in obj_lists:
            if type(obj_list) == list:
                if hasattr(obj_list[i][j], 'get_feature_names_out'):
                    n = obj_list[i][j].get_feature_names_out()  
                elif hasattr(obj_list[i][j][0], 'columns'):
                    n = obj_list[i][j][0].columns
                else:
                    n = np.arange(obj_list[i][j][0].shape[-1])
                for k in n:
                    names.append('{}__{}'.format(name, k))
            else:
                for k in X:
                    names.append('{}__{}'.format(name, k))
        return names
    def get_transform(i, j, v, idx):
        v = list(v)
        l = list()
        for name, obj_list, X in obj_lists:
            if isinstance(obj_list, list) :
                if hasattr(obj_list[i][j], 'transform'):
                    l.append(obj_list[i][j].transform(v[0][X]))
                else:
                    l.append(obj_list[i][j][idx])
            else: 
                if type(v[0]) == pd.DataFrame:
                    l.append(v[0][X])
                else:
                    l.append(v[0][:, X])
        if type(l[0]) == pd.DataFrame:
            X = pd.concat([l.rename(columns=lambda x: name + '__' + x) for l, (name, _, _) in zip(l, obj_lists)], axis=1)
        else:
            X = np.hstack(l)
        if len(v) == 1:
            return X
        else:
            return X, v[1] if len(v) == 2 else v[1:]

    for i, (train_list, valid) in enumerate(split(train_idx_list, valid_idx_list, *args, split_train=split_train)):
        for j, (train, train_v) in enumerate(train_list):
            names = get_feature_names(i, j)
            train_v_transform = get_transform(i, j, train_v, 1) if train_v is not None else None
            yield i, j, names, get_transform(i, j, train, 0), train_v_transform, get_transform(i, j, valid, 2)
            
def split_and_transform_store(obj_lists, train_idx_list, valid_idx_list, *args, split_train=False):
    l1 = list()
    i_ = -1
    for i, j, n, X_train, X_train_v, X_valid in split_and_transform(
            obj_lists, train_idx_list, valid_idx_list, *args, split_train = True
    ):
        if i_ != i:
            l1.append(list())
        l1[-1].append(
            (X_train, X_train_v, X_valid)
        )
        i_ = i
    return l1

In [204]:
pca_values = split_and_transform_store([
    ('pca', pca_list, X_num)
], train_idx_list, valid_idx_list, df_train[X_all], split_train = True)
pca_values

[[(        pca__pca0  pca__pca1  pca__pca2  pca__pca3  pca__pca4  pca__pca5
   id                                                                      
   138907  -1.625568  -0.720244  -0.505836  -1.044785   1.783526   0.142959
   251061  -0.730669   0.109349  -1.269454  -0.898767   0.380521   0.015633
   108883   1.098706   1.163701  -0.517042   1.671697   0.885004  -0.097822
   421891  -0.526545   0.345860  -0.793755  -1.206155  -0.396592  -0.047605
   302478  -0.677216   0.450531   0.810389  -0.429099   0.386358  -0.325950
   ...           ...        ...        ...        ...        ...        ...
   470641   1.349882   0.043285  -0.001735  -0.295368   0.407611  -0.118700
   170626  -0.423096   0.460783   0.414791  -1.029564   0.557638  -0.467625
   493480  -0.072579   0.191544   1.633167  -0.047257  -0.731524  -0.079870
   415113   3.116569  -0.119307  -0.233893  -0.898333  -0.808953  -0.119866
   409290  -0.330161  -0.737932  -0.556850  -0.255160   0.583255  -0.235990
   
   [4276

In [205]:
for i, j, n, (X_a, X_b), b, c in split_and_transform(
        [
            ('num', 'passthrough', X_num),
            ('cat', 'passthrough', X_cat),
            ('std', std_list, X_num),
            ('pca', pca_values, X_num),
        ], train_idx_list, valid_idx_list, df_train[X_all], df_train[y], split_train = True
    ):
    print(len(n), X_a, X_b, b, c)

23         num__annual_income  num__debt_to_income_ratio  num__credit_score  \
id                                                                         
138907        25482.740234                      0.046                771   
251061        14653.769531                      0.099                713   
108883        52467.761719                      0.271                657   
421891        27400.689453                      0.079                690   
302478        71005.390625                      0.093                697   
...                    ...                        ...                ...   
470641        47481.230469                      0.113                636   
170626        59136.589844                      0.070                685   
493480        90142.117188                      0.088                667   
415113        37758.429688                      0.084                554   
409290        26693.779297                      0.093                692   

        

23         num__annual_income  num__debt_to_income_ratio  num__credit_score  \
id                                                                         
138365        45462.398438                      0.094                626   
250661        48551.820312                      0.085                615   
108304        24763.349609                      0.213                721   
421429        57123.210938                      0.035                645   
301843        38008.398438                      0.085                713   
...                    ...                        ...                ...   
469936        28597.250000                      0.041                750   
170262        69639.617188                      0.092                705   
493028        63626.121094                      0.092                706   
414586         7991.229980                      0.075                654   
408736        23113.119141                      0.096                733   

        

In [193]:
for i, j, n, a, b, c in split_and_transform(
        [
            ('std', std_list, X_num),
        ], train_idx_list, valid_idx_list, df_train[X_all], split_train = True
    ):
    print(i, j, len(n), a, b, c)

0 0 6         std__annual_income  std__debt_to_income_ratio  std__credit_score  \
id                                                                         
138907           -0.851332                  -1.089396           1.624995   
251061           -1.256891                  -0.316139           0.579265   
108883            0.159293                   2.193300          -0.430405   
421891           -0.779502                  -0.607934           0.164579   
302478            0.853551                  -0.403678           0.290788   
...                    ...                        ...                ...   
470641           -0.027460                  -0.111882          -0.809031   
170626            0.409049                  -0.739242           0.074430   
493480            1.570246                  -0.476626          -0.250107   
415113           -0.391591                  -0.534985          -2.287477   
409290           -0.805977                  -0.403678           0.200639   

     

In [196]:
b

,std__annual_income,std__debt_to_income_ratio,std__credit_score,std__loan_amount,std__interest_rate,std__grade_subgrade_no
id,,,,,,
564861,-0.073325,-1.265326,1.264774,0.597407,-0.192226,-1.558253
472137,0.849818,-0.988092,-0.250989,-1.143818,-1.500992,0.397031
3899,0.298797,0.952548,1.228684,1.218469,-1.242225,-1.753781
209458,4.086154,2.324128,1.336953,-0.170995,-1.137722,-1.362725
503911,-0.750164,-0.594128,1.770028,1.174754,-0.709761,-1.753781
...,...,...,...,...,...,...
531288,-0.178939,-0.346076,-0.250989,0.812140,-0.147440,0.397031
445754,-0.012579,-0.506580,-1.243452,1.747563,1.151373,0.592559
247275,0.284006,1.944755,1.860252,-0.029998,-0.316634,-1.167196


In [72]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

[([(        annual_income  debt_to_income_ratio  credit_score   loan_amount  \
id                                                                        
155912   24490.759766                 0.084           689   5284.439941   
209343   48883.128906                 0.046           624   9142.129883   
392510   33248.949219                 0.069           667  12961.580078   
153135   50937.878906                 0.067           728  23351.890625   
274644   14226.280273                 0.064           741  12661.700195   
...               ...                   ...           ...           ...   
336308   53977.089844                 0.096           596   8354.980469   
387786   10603.269531                 0.138           718  15449.080078   
16380    52320.039062                 0.122           628  13597.519531   
255063   54593.730469                 0.222           699  11443.610352   
156090   10374.799805                 0.091           585   6127.660156   

        interest_rat

In [172]:
def eval_cb(cb_params, X_num, X_cat, sp = ss, eval_size = 0.0):
    scores = list()
    prds = list()
    clfs = list()
    X_all = list(set(X_num).union(set(X_cat)))
    eval_set = None
    for train_idx, valid_idx in sp.split(df_train[X_all], df_train[y]):
        clf_cb = cb.CatBoostClassifier(
            **cb_params, cat_features = X_cat, verbose = 0
        )
        df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
        # lr_prb = cross_val_predict(clf_lr, df_cv_train[X_lr], df_cv_train[y], cv = skf, method = 'predict_proba')[:, 1]
        # clf_cb.fit(df_cv_train[X_all].assign(lr_prb = lr_prb), df_cv_train[y])
        if eval_size > 0:
            df_cv_train, df_eval = train_test_split(
                df_cv_train, test_size = eval_size, random_state = 123, stratify = df_train[y]
            )
            eval_set = [(df_eval[X_all], df_eval[y])]
        clf_cb.fit(df_cv_train[X_all], df_cv_train[y], eval_set=eval_set)
        prd = pd.Series(clf_cb.predict_proba(df_valid[X_all])[:, 1], index = df_valid.index)
        prds.append(prd)
        scores.append(
            roc_auc_score(df_valid[y], prd)
        )
        clfs.append(clf_cb)
    return np.mean(scores), np.std(scores), pd.concat(prds).sort_index().values, clfs, X_all

In [181]:
results = eval_cb(
    {
        'n_estimators': 2000,
        'task_type': 'CPU',
        'max_depth': 6,
        'learning_rate':0.1,
        'border_count': 254, # border_count 128 -> 254
        'subsample': 0.75 # subsample: 0.75 성능 향상
    }, 
    ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate'] + ['grade_subgrade_no'],
    ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose'],
)
results[0]

0.9248421854596397

In [182]:
pd.DataFrame(
    results[3][0].get_feature_importance(type = 'Interaction'), columns = ['v1', 'v2', 'importance']
).assign(
    v1 = lambda x: x['v1'].astype('int').apply(lambda x: results[-1][x]),
    v2 = lambda x: x['v2'].astype('int').apply(lambda x: results[-1][x])
).iloc[:10]

,v1,v2,importance
0,debt_to_income_ratio,credit_score,9.104483
1,debt_to_income_ratio,loan_amount,7.272959
2,debt_to_income_ratio,annual_income,6.670799
3,debt_to_income_ratio,interest_rate,6.178732
4,credit_score,loan_amount,4.848106
5,credit_score,interest_rate,4.718051
6,credit_score,annual_income,4.226164
7,debt_to_income_ratio,employment_status,4.046364
8,interest_rate,loan_amount,3.582667
9,loan_amount,annual_income,3.558486


In [152]:
pd.DataFrame?